In [50]:
import pandas as pd
import VCF
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [51]:
egfr_singleHit_00 = VCF.dataframe('artificalVCF/egfr_singleHit_00.vcf')
egfr_singleHit_00

FileNotFoundError: [Errno 2] No such file or directory: 'artificalVCF/egfr_singleHit_00.vcf'

In [52]:
t = pd.read_csv('/Users/lincoln.harris/code/cerebra/cerebra/wrkdir/geneCellMutationCounts_artifical_TEST.csv')
t

,Unnamed: 0,BRAF,EGFR,KRAS
0,artificalVCF/B1_1000,1,0,0
1,artificalVCF/B1_1001,1,0,0
2,artificalVCF/B1_1002,1,0,0
3,artificalVCF/B1_1003,1,0,0
4,artificalVCF/B1_1004,1,0,0
5,artificalVCF/E1_1000,0,1,0
6,artificalVCF/E1_1001,0,1,0
7,artificalVCF/E1_1002,0,1,0
8,artificalVCF/E1_1003,0,1,0
9,artificalVCF/E1_1004,0,1,0


In [35]:
hg38_gtf = pd.read_csv('/Users/lincoln.harris/code/cerebra/cerebra/wrkdir/hg38-plus.gtf', delimiter = '\t', header = None)
hg38_gtf

,0,1,2,3,4,5,6,7,8
0,chr1,unknown,exon,11874,12227,.,+,.,"gene_id ""DDX11L1""; gene_name ""DDX11L1""; transc..."
1,chr1,unknown,exon,12613,12721,.,+,.,"gene_id ""DDX11L1""; gene_name ""DDX11L1""; transc..."
2,chr1,unknown,exon,13221,14409,.,+,.,"gene_id ""DDX11L1""; gene_name ""DDX11L1""; transc..."
3,chr1,unknown,exon,14362,14829,.,-,.,"gene_id ""WASH7P""; gene_name ""WASH7P""; transcri..."
4,chr1,unknown,exon,14970,15038,.,-,.,"gene_id ""WASH7P""; gene_name ""WASH7P""; transcri..."
5,chr1,unknown,exon,15796,15947,.,-,.,"gene_id ""WASH7P""; gene_name ""WASH7P""; transcri..."
6,chr1,unknown,exon,16607,16765,.,-,.,"gene_id ""WASH7P""; gene_name ""WASH7P""; transcri..."
7,chr1,unknown,exon,16858,17055,.,-,.,"gene_id ""WASH7P""; gene_name ""WASH7P""; transcri..."
8,chr1,unknown,exon,17233,17368,.,-,.,"gene_id ""WASH7P""; gene_name ""WASH7P""; transcri..."
9,chr1,unknown,exon,17369,17436,.,-,.,"gene_id ""MIR6859-3""; gene_name ""MIR6859-3""; tr..."


In [36]:
returnStr = str(hg38_gtf.iloc[0][8])     # keep just the gene name / metadata col
returnStr = returnStr.split(';')[1]
returnStr = returnStr.strip(' gene_name')
returnStr = returnStr.strip(' ')
returnStr = returnStr.strip('"')
returnStr

'DDX11L1'

In [37]:
def get_gene_name(posString):
    """ return the gene name from a given genome position string
       (ie. '1:21890111-21890111'), by querying the hg38-plus.gtf """

    chrom = posString.split(':')[0] # work on posString
    posString_remove = posString.split(':')[1]
    lPosition = posString_remove.split('-')[0] 
    rPosition = posString_remove.split('-')[1] 

    # work on hg38_gtf
    chromStr = 'chr' + str(chrom)
    hg38_gtf_filt = hg38_gtf.where(hg38_gtf[0] == chromStr).dropna()
    hg38_gtf_filt = hg38_gtf_filt.where(hg38_gtf_filt[3] <= int(lPosition)).dropna() # lPos good
    hg38_gtf_filt = hg38_gtf_filt.where(hg38_gtf_filt[4] >= int(rPosition)).dropna() # rPos good

    try:
        returnStr = str(hg38_gtf_filt.iloc[0][8])	# keep just the gene name / metadata col
        returnStr = returnStr.split(';')[1]
        returnStr = returnStr.strip(' gene_name')
        returnStr = returnStr.strip(' ')
        returnStr = returnStr.strip('"')
    except IndexError:
        returnStr = ''

    return returnStr


In [38]:
# test case 
get_gene_name('1:21890111-21890111')

'HSPG2'

In [40]:
# not quite 
get_gene_name('7:55182350-55182350')

'EGFR-AS1'

In [41]:
# should be EGFR
get_gene_name('7:55152340-55152340')

''

In [42]:
# should be BRAF
get_gene_name('7:140900700-140900700')

''

In [46]:
# why does this one work? 
get_gene_name('7:55191821-55191821')

'EGFR'

In [44]:
get_gene_name('7:55181377-55181379')

'EGFR-AS1'

In [45]:
get_gene_name('7:140753335-140753337')

'BRAF'

In [47]:
get_gene_name('7:140753336-140753336')

'BRAF'

In [48]:
get_gene_name('12:25245351-25245351')

'KRAS'

In [53]:
# get empty region 
get_gene_name('14:1000-1001')

''